In [1]:
using DataFrames, MLJ

In [2]:
X, y = @load_boston;
X = DataFrame(X)
y = Vector(y);

In [10]:
rr_model = @load RidgeRegressor pkg=MLJLinearModels add=true
rr_pipe = @pipeline Standardizer rr_model

import MLJLinearModels ✔
import MLJLinearModels ✔
import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\julio\.julia\packages\MLJModels\GKDnU\src\loading.jl:168
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\julio\.julia\packages\MLJModels\GKDnU\src\loading.jl:168
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\julio\.julia\packages\MLJModels\GKDnU\src\loading.jl:168


Pipeline284(
    standardizer = Standardizer(
            features = Symbol[],
            ignore = false,
            ordered_factor = false,
            count = false),
    ridge_regressor = RidgeRegressor(
            lambda = 1.0,
            fit_intercept = true,
            penalize_intercept = false,
            solver = nothing))

In [4]:
fieldnames(rr_model)

(:lambda, :fit_intercept, :penalize_intercept, :solver)

In [9]:
rf_model = @load RandomForestRegressor pkg=DecisionTree add=true
rf_pipe = @pipeline Standardizer rf_model

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\julio\.julia\packages\MLJModels\GKDnU\src\loading.jl:168


Pipeline290(
    standardizer = Standardizer(
            features = Symbol[],
            ignore = false,
            ordered_factor = false,
            count = false),
    random_forest_regressor = RandomForestRegressor(
            max_depth = -1,
            min_samples_leaf = 1,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = -1,
            n_trees = 10,
            sampling_fraction = 0.7,
            pdf_smoothing = 0.0,
            rng = Random._GLOBAL_RNG()))

In [37]:
r = [
    range(rf_pipe, :(random_forest_regressor.max_depth), lower=1, upper=5),
    range(rf_pipe, :(random_forest_regressor.n_trees), lower=10, upper=50)
    ]

tuned2 = TunedModel(model=rf_pipe,
                    range=r,
                    measure=rms);
m = machine(tuned2, X, y) |> fit!
fitted_params(m).best_model

┌ Info: Training Machine{DeterministicTunedModel{Grid,…},…}.
└ @ MLJBase C:\Users\julio\.julia\packages\MLJBase\QXObv\src\machines.jl:403
┌ Info: Attempting to evaluate 50 models.
└ @ MLJTuning C:\Users\julio\.julia\packages\MLJTuning\bjRHJ\src\tuned_models.jl:680
Evaluating over 50 metamodels:   4%[=>                       ]  ETA: 0:00:26

Evaluating over 50 metamodels: 100%[=========================] Time: 0:00:02


Pipeline290(
    standardizer = Standardizer(
            features = Symbol[],
            ignore = false,
            ordered_factor = false,
            count = false),
    random_forest_regressor = RandomForestRegressor(
            max_depth = 5,
            min_samples_leaf = 1,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = -1,
            n_trees = 23,
            sampling_fraction = 0.7,
            pdf_smoothing = 0.0,
            rng = Random._GLOBAL_RNG()))

In [11]:
tuned_model = TunedModel(
    models=[rr_pipe, rf_pipe],
    measure=rms,
    train_best=true
)

DeterministicTunedModel(
    model = Pipeline297(
            standardizer = Standardizer,
            ridge_regressor = RidgeRegressor),
    tuning = Explicit(),
    resampling = Holdout(
            fraction_train = 0.7,
            shuffle = false,
            rng = Random._GLOBAL_RNG()),
    measure = RootMeanSquaredError(),
    weights = nothing,
    operation = nothing,
    range = DeterministicComposite[Pipeline297, Pipeline290],
    selection_heuristic = MLJTuning.NaiveSelection(nothing),
    train_best = true,
    repeats = 1,
    n = nothing,
    acceleration = CPU1{Nothing}(nothing),
    acceleration_resampling = CPU1{Nothing}(nothing),
    check_measure = true,
    cache = true)

In [13]:
m = machine(tuned_model, X, y) |> fit!

┌ Info: Training Machine{DeterministicTunedModel{Explicit,…},…}.
└ @ MLJBase C:\Users\julio\.julia\packages\MLJBase\QXObv\src\machines.jl:403
┌ Info: Attempting to evaluate 2 models.
└ @ MLJTuning C:\Users\julio\.julia\packages\MLJTuning\bjRHJ\src\tuned_models.jl:680


Machine{DeterministicTunedModel{Explicit,…},…} trained 1 time; caches data
  args: 
    1:	Source @961 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @390 ⏎ `AbstractVector{Continuous}`


In [15]:
fitted_params(m)

(best_model = Pipeline290,
 best_fitted_params = (random_forest_regressor = (forest = Ensemble of Decision Trees
Trees:      10
Avg Leaves: 247.7
Avg Depth:  17.0,),
                       standardizer = Dict(:Zn => (11.363636363636363, 23.322452994515135), :Rad => (9.549407114624506, 8.707259384239368), :Tax => (408.2371541501976, 168.537116054959), :LStat => (12.653063241106722, 7.141061511348569), :Rm => (6.284634387351778, 0.7026171434153233), :Age => (68.57490118577076, 28.148861406903617), :NOx => (0.5546950592885372, 0.11587767566755595), :Indus => (11.136778656126486, 6.8603529408975845), :Dis => (3.795042687747036, 2.105710126627611), :PTRatio => (18.45553359683795, 2.1649455237144406)…),
                       machines = Machine[Machine{Standardizer,…}, Machine{RandomForestRegressor,…}],
                       fitted_params_given_machine = OrderedCollections.LittleDict{Any, Any, Vector{Any}, Vector{Any}}(Machine{Standardizer,…} => Dict(:Zn => (11.363636363636363, 23.322452994

In [21]:
r = report(m)
@show r.best_history_entry.measurement;

r.best_history_entry.measurement = [7.69887765231161]


In [27]:
r.best_model

Pipeline290(
    standardizer = Standardizer(
            features = Symbol[],
            ignore = false,
            ordered_factor = false,
            count = false),
    random_forest_regressor = RandomForestRegressor(
            max_depth = -1,
            min_samples_leaf = 1,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = -1,
            n_trees = 10,
            sampling_fraction = 0.7,
            pdf_smoothing = 0.0,
            rng = Random._GLOBAL_RNG()))

In [26]:
r.best_report

(random_forest_regressor = (),
 standardizer = (features_fit = [:Zn, :Rad, :Tax, :LStat, :Rm, :Age, :NOx, :Indus, :Dis, :PTRatio, :Crim, :Black],),
 machines = Machine[Machine{Standardizer,…}, Machine{RandomForestRegressor,…}],
 report_given_machine = OrderedCollections.LittleDict{Any, Any, Vector{Any}, Vector{Any}}(Machine{Standardizer,…} => (features_fit = [:Zn, :Rad, :Tax, :LStat, :Rm, :Age, :NOx, :Indus, :Dis, :PTRatio, :Crim, :Black],), Machine{RandomForestRegressor,…} => NamedTuple()),)